### Code below all in one cell for ease of running whilst operation

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  3 12:49:01 2025

@author: Oonagh
"""

## BioEM EOG Project ##

import serial
import time
import keyboard   # pip install keyboard

# ======== CONFIG ========

SERIAL_PORT = 'COM4'
BAUDRATE = 9600

BLINK_THRESHOLD_DU = 400      # tuned threshold
BASELINE_ALPHA = 0.01         # EMA speed

REFRACTORY_MS = 200           # minimum time between blinks
SEQ_WINDOW_MS = 900           # grouping window for multiple blinks
DOUBLE_DECISION_DELAY_MS = 450  # wait for possible 3rd blink

# =========================


def send_play_pause():
    print(">> PLAY/PAUSE")
    keyboard.send("play/pause media")


def send_next_track():
    print(">> NEXT TRACK")
    keyboard.send("next track")


def main():
    # Open serial
    ser = serial.Serial(SERIAL_PORT, BAUDRATE, timeout=1)

    baseline = None
    last_blink_time = 0.0
    in_blink = False

    blink_times = []
    is_playing = False

    pending_action = None

    print("Connected to", SERIAL_PORT)
    print("Blink commands:")
    print("  2 blinks: play/pause")
    print("  3 blinks (while playing): skip track\n")

    while True:
        line = ser.readline()
        if not line:
            # no data
            now = time.time()
            continue

        try:
            text = line.decode('ascii', errors='ignore').strip()
            parts = text.split()
            if not parts:
                continue
            v = int(parts[-1])  # last column is EOG
        except Exception:
            continue

        now = time.time()

        # Initialize baseline first time
        if baseline is None:
            baseline = v

        # baseline update (EMA)
        baseline = (1.0 - BASELINE_ALPHA) * baseline + BASELINE_ALPHA * v
        dev = v - baseline
        mag = abs(dev)

        # ---- Blink detection ----
        if mag > BLINK_THRESHOLD_DU:
            # possible blink
            if not in_blink and (now - last_blink_time) * 1000.0 > REFRACTORY_MS:
                last_blink_time = now
                in_blink = True

                blink_times.append(now)

                # remove old blinks
                cutoff = now - (SEQ_WINDOW_MS / 1000.0)
                blink_times = [t for t in blink_times if t >= cutoff]

                num_recent = len(blink_times)
                print(f"Blink detected. Recent blinks: {num_recent}")

                # sequence logic
                if num_recent == 2:
                    pending_action = {'type': 'double', 't': now}

                elif num_recent >= 3:
                    # triple
                    pending_action = None
                    if is_playing:
                        send_next_track()
                    else:
                        print("Triple blink but music not playing — ignoring")

                    blink_times = []

        else:
            # end of blink
            if in_blink and mag < BLINK_THRESHOLD_DU * 0.5:
                in_blink = False

        # ---- Committing double-blink ----
        if pending_action is not None:
            dt_ms = (now - pending_action['t']) * 1000.0
            if dt_ms > DOUBLE_DECISION_DELAY_MS:
                # confirm double
                if is_playing:
                    send_play_pause()
                    is_playing = False
                    print("State: PAUSED")
                else:
                    send_play_pause()
                    is_playing = True
                    print("State: PLAYING")

                pending_action = None
                blink_times = []

        time.sleep(0.001)


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nExiting...")


 ### Calibration

In [28]:
import serial
import time
import numpy as np

SERIAL_PORT = 'COM4'   
BAUDRATE = 9600

ser = serial.Serial(SERIAL_PORT, BAUDRATE, timeout=1)
print("Serial connected on", SERIAL_PORT)


Serial connected on COM4


In [29]:
BASELINE_ALPHA = 0.01  # same as in main code

def calibrate_blinks(duration_s=15):
    baseline = None
    mags = []

    print("Calibration starting…")
    print("0–5 s: keep eyes still, relaxed (no blinking)")
    print("5–15 s: make several strong intentional blinks")
    print(f"Recording for {duration_s} seconds...\n")

    t0 = time.time()
    n_lines = 0

    while time.time() - t0 < duration_s:
        line = ser.readline()
        n_lines += 1

        if not line:
            continue

        try:
            text = line.decode('ascii', errors='ignore').strip()
            parts = text.split()
            if len(parts) == 0:
                continue
            # take the LAST number on the line -> the EOG sample
            v = int(parts[-1])
        except Exception:
            continue

        if baseline is None:
            baseline = v
        else:
            baseline = (1.0 - BASELINE_ALPHA) * baseline + BASELINE_ALPHA * v

        mag = abs(v - baseline)
        mags.append(mag)

    print("Total lines read from serial:", n_lines)
    print("Valid EOG samples used:", len(mags))

    if len(mags) == 0:
        print("\n❌ No valid samples collected! (Check parsing and Arduino output.)\n")
        return None

    import numpy as np
    mags_arr = np.array(mags)
    noise95 = np.percentile(mags_arr, 95)
    blink99 = np.percentile(mags_arr, 99.5)
    blink_max = mags_arr.max()

    print("\n--- Calibration Results ---")
    print("Samples:", len(mags))
    print(f"Noise (95th percentile):        {noise95:.1f}")
    print(f"Blink strength (99.5th perc.):  {blink99:.1f}")
    print(f"Maximum magnitude:              {blink_max:.1f}")

    lower = noise95 + 5
    mid   = (noise95 + blink99) / 2
    upper = max(noise95 * 1.5, noise95 + 10)

    print("\nSuggested BLINK_THRESHOLD_DU range:")
    print(f"  Low  ~ {lower:.1f}")
    print(f"  Mid  ~ {mid:.1f}")
    print(f"  High ~ {upper:.1f}")
    print("\nPick one of these and set BLINK_THRESHOLD_DU to it in your main script.\n")

    return {
        "noise95": noise95,
        "blink99": blink99,
        "blink_max": blink_max,
        "thr_lower": lower,
        "thr_mid": mid,
        "thr_upper": upper,
    }

print("Calibration function ready.")


Calibration function ready.


In [30]:
calib = calibrate_blinks(duration_s=15)
calib

Calibration starting…
0–5 s: keep eyes still, relaxed (no blinking)
5–15 s: make several strong intentional blinks
Recording for 15 seconds...

Total lines read from serial: 3696
Valid EOG samples used: 3696

--- Calibration Results ---
Samples: 3696
Noise (95th percentile):        265.6
Blink strength (99.5th perc.):  637.9
Maximum magnitude:              728.8

Suggested BLINK_THRESHOLD_DU range:
  Low  ~ 270.6
  Mid  ~ 451.7
  High ~ 398.4

Pick one of these and set BLINK_THRESHOLD_DU to it in your main script.



{'noise95': np.float64(265.5795645229681),
 'blink99': np.float64(637.9127905637527),
 'blink_max': np.float64(728.7952183277055),
 'thr_lower': np.float64(270.5795645229681),
 'thr_mid': np.float64(451.7461775433604),
 'thr_upper': np.float64(398.36934678445215)}